I will start by encoding my data.


In [755]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("final_cleaned_home_price_data.csv")

I will NOT  scale net area and gross area now to avoid data leakage


I will do one hot encoding on Occupancy status, Investment elligibility, Floor location, and Title deed status.

I will also save these original values in a pkl file to retrieve later for user input

In [756]:
#for investment eligibility
from sklearn.preprocessing import OneHotEncoder

ohe_investment = OneHotEncoder(
    handle_unknown='ignore',
    sparse_output=False
)

ohe_investment.fit(df[['Investment_Eligibility']])

encoded = ohe_investment.transform(df[['Investment_Eligibility']])
encoded_df = pd.DataFrame(
    encoded,
    columns=ohe_investment.get_feature_names_out(['Investment_Eligibility']),
    index=df.index
)

df = df.drop(columns=['Investment_Eligibility'])
df = pd.concat([df, encoded_df], axis=1)


with open("ohe_investment.pkl", "wb") as f:
    pickle.dump(ohe_investment, f)


In [757]:
#for title deed

ohe_deed = OneHotEncoder(
    handle_unknown='ignore',
    sparse_output=False
)

ohe_deed.fit(df[['Title_Deed_Status']])

encoded = ohe_deed.transform(df[['Title_Deed_Status']])
encoded_df = pd.DataFrame(
    encoded,
    columns=ohe_deed.get_feature_names_out(['Title_Deed_Status']),
    index=df.index
)

df = df.drop(columns=['Title_Deed_Status'])
df = pd.concat([df, encoded_df], axis=1)


with open("ohe_deed.pkl", "wb") as f:
    pickle.dump(ohe_deed, f)



In [758]:
#for floor location

ohe_floor_location = OneHotEncoder(
    handle_unknown = 'ignore',
    sparse_output = False    
)
ohe_floor_location.fit(df[['Floor_Location']])
encoded = ohe_floor_location.transform(df[['Floor_Location']])

encoded_df = pd.DataFrame(
    encoded,
    columns=ohe_floor_location.get_feature_names_out(['Floor_Location']),
    index=df.index
)

df = df.drop(columns=['Floor_Location'])
df = pd.concat([df, encoded_df], axis=1)


with open("ohe_floor_location.pkl", "wb") as f:
    pickle.dump(ohe_floor_location, f)


In [759]:
#for occupancy status
ohe_occupancy = OneHotEncoder(
    handle_unknown='ignore',
    sparse_output=False
)

ohe_occupancy.fit(df[['Occupancy_Status']])

encoded = ohe_occupancy.transform(df[['Occupancy_Status']])
encoded_df = pd.DataFrame(
    encoded,
    columns=ohe_occupancy.get_feature_names_out(['Occupancy_Status']),
    index=df.index
)

df = df.drop(columns=['Occupancy_Status'])
df = pd.concat([df, encoded_df], axis=1)


with open("ohe_occupancy.pkl", "wb") as f:
    pickle.dump(ohe_occupancy, f)
    

I will do ordinal encoding on Room count, Building age, Heating type, and Bathroom count  

In [760]:
#for room count
from sklearn.preprocessing import OrdinalEncoder

ordinal_room = {1:0,2:1,2.5:2,3:3,3.5:4,4:5,
                                     4.5:6,5:7,5.5:8,6:9,7:10,8:11,
                                     9:12,10:13,11:14,12:15}

df['Room_Count_ordinal'] = df['Room_Count'].map(ordinal_room)
df = df.drop(columns=['Room_Count'])


with open("ordinal_room.pkl", "wb") as f:
    pickle.dump(ordinal_room, f)

In [761]:
#for building age
ordinal_age = {'0 (Yeni)':0, '1':1,'2':2,'3':3,'4':4,'5-10':5,'11-15':6,'16-20':7,'21 Ve Üzeri':8}
df['Building_Age'] = df['Building_Age'].astype(str)
df['Building_Age'] = df['Building_Age'].str.strip()

df['Building_Age_ordinal'] = df['Building_Age'].map(ordinal_age)
df = df.drop(columns=['Building_Age'])

with open("ordinal_age.pkl", "wb") as f:
    pickle.dump(ordinal_age, f)
    # Assuming you are checking the 'Building_Age' column and using the 'ordinal_age' map





In [762]:
#for heating type
ordinal_heating = {'Isıtma Yok':0,'Sobalı':1,'Merkezi Kömür':2,'Doğalgaz Sobalı':3,
                                              'Diğer':4,'Güneş Enerjisi':5,'Kat Kaloriferi':6,'Merkezi Doğalgaz':7,
                                              'Merkezi (Pay Ölçer)':8,'Kombi Doğalgaz':9,'Klimalı':10,'Yerden Isıtma':11,'Jeotermal':12}
df['Heating_Type'] = df['Heating_Type'].astype(str)
df['Heating_Type'] = df['Heating_Type'].str.strip()

df['Heating_Type_ordinal'] = df['Heating_Type'].map(ordinal_heating)
df = df.drop(columns=['Heating_Type'])

with open("ordinal_heating.pkl", "wb") as f:
    pickle.dump(ordinal_heating, f)


In [763]:
#for bathroom count

ordinal_bathroom = {'0.0':0,'1.0':1,'2.0':2,'3.0':3,'4.0':4,'5.0':5,'6.0':6}
df['Bathroom_Count'] = df['Bathroom_Count'].astype(str)
df['Bathroom_Count'] = df['Bathroom_Count'].str.strip()


df['Bathroom_Count_ordinal'] = df['Bathroom_Count'].map(ordinal_bathroom)
df = df.drop(columns=['Bathroom_Count'])

with open("ordinal_bathroom.pkl", "wb") as f:
    pickle.dump(ordinal_bathroom, f)



I will now split my data

In [764]:
from sklearn.model_selection import train_test_split

x = df.drop(columns = ['Price'])
y = df['Price']

x_train, x_temp, y_train, y_temp = train_test_split(
    x,
    y,
    test_size = 0.3,
    random_state=100,
)

In [765]:
x_val, x_test, y_val, y_test = train_test_split(
    x_temp,
    y_temp,
    test_size = 0.5,
    random_state = 100
)

I will now scale my columns, and apply target smoothing on city BEFORE training my model

In [766]:
from sklearn.preprocessing import StandardScaler

x_train_numerical = x_train[['Net_Area', 'Gross_Area']]

scaled_net_gross = StandardScaler()
scaled_net_gross.fit(x_train_numerical)

with open('scaled_net_gross.pkl', 'wb') as file:
    pickle.dump(scaled_net_gross, file)

I will apply target smoothing on my city column

In [767]:
from category_encoders import TargetEncoder


target_city = TargetEncoder(
    min_samples_leaf=20, 
    smoothing=10.0 )

target_city.fit(x_train[['City']], y_train)

x_train_city = target_city.transform(x_train[['City']]).values


with open('target_city.pkl', 'wb') as file:
    pickle.dump(target_city, file)




In [768]:
import pickle
from sklearn.preprocessing import StandardScaler

scaled_city = StandardScaler()
scaled_city.fit(x_train_city)
with open('scaled_city.pkl', 'wb') as file:
    pickle.dump(scaled_city, file)

In [769]:
from sklearn.pipeline import Pipeline 

city_pipeline_steps = [
    ('target_smooth', TargetEncoder(min_samples_leaf=20, smoothing=10.0)),
    ('scaler', StandardScaler())
]

city_transform_pipeline = Pipeline(city_pipeline_steps)

city_transform_pipeline.fit(x_train[['City']], y_train)

#x_test_city_transformed = city_transform_pipeline.transform(x_test[['City']])

x_train = x_train.drop('City', axis=1)
x_train['City_Target_Scaled'] = pd.DataFrame(x_train_city_transformed, 
                                            index=x_train.index)

x_test = x_test.drop('City', axis=1)
x_test['City_Target_Scaled'] = pd.DataFrame(x_test_city_transformed, 
                                          index=x_test.index)
                                          
with open('target_scaled_city', 'wb') as file:
    pickle.dump(city_transform_pipeline, file)

I will transform my test and train data before merging all pkl files together in a matrix

for train data:

In [770]:
"""import numpy as np
from sklearn.linear_model import LinearRegression

def load_transformer(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)
    

net_gross_scaler = load_transformer('scaled_net_gross.pkl')


columns_to_transform = ['Net_Area','Gross_Area']
x_train[columns_to_transform] = net_gross_scaler.transform(x_train[columns_to_transform])
x_test[columns_to_transform] = net_gross_scaler.transform(x_test[columns_to_transform])"""


"import numpy as np\nfrom sklearn.linear_model import LinearRegression\n\ndef load_transformer(filename):\n    with open(filename, 'rb') as file:\n        return pickle.load(file)\n\n\nnet_gross_scaler = load_transformer('scaled_net_gross.pkl')\n\n\ncolumns_to_transform = ['Net_Area','Gross_Area']\nx_train[columns_to_transform] = net_gross_scaler.transform(x_train[columns_to_transform])\nx_test[columns_to_transform] = net_gross_scaler.transform(x_test[columns_to_transform])"

In [ ]:
def load_transformer(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

city_transform_pipeline = load_transformer('target_scaled_city')
net_gross_scaler = load_transformer('scaled_net_gross.pkl')

x_val_city_transformed = city_transform_pipeline.transform(x_val[['City']])

x_val = x_val.drop('City', axis=1)

x_val['City_Target_Scaled'] = pd.Series(
    x_val_city_transformed[:, 0], 
    index=x_val.index
)

columns_to_transform = ['Net_Area', 'Gross_Area']
x_val[columns_to_transform] = net_gross_scaler.transform(x_val[columns_to_transform])

In [772]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)
X_test_scaled = scaler.transform(x_val)

In [773]:
import numpy as np

y_train = np.log1p(y_train)
y_test = np.log1p(y_test)
y_val = np.log1p(y_val)

In [774]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [775]:
from sklearn.metrics import mean_squared_error, r2_score

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"\n## Evaluation Metrics ##")
print(f"R-squared (R2): {r2:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")




## Evaluation Metrics ##
R-squared (R2): 0.6225
Mean Squared Error (MSE): 0.1327
Root Mean Squared Error (RMSE): 0.3642


In [776]:
y_val_pred_orig_scale = np.expm1(y_val)
y_pred2 = model.predict(x_val)
r2 = r2_score(y_val, y_pred2)
mse = mean_squared_error(y_val, y_pred2)
rmse = np.sqrt(mse)
print(f"\n## Evaluation Metrics ##")
print(f"R-squared (R2): {r2:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")



## Evaluation Metrics ##
R-squared (R2): 0.2140
Mean Squared Error (MSE): 0.3045
Root Mean Squared Error (RMSE): 0.5518


In [777]:

print('model score: ' ,model.score(x_train,y_train))



model score:  0.6446358041678226


strong overfitting, not sure how to combat it :'(